In [2]:
#Optimizacion Bayesiana de hiperparametros de  rpart
#funciona automaticamente con EXPERIMENTOS
#va generando incrementalmente salidas para kaggle

#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

library("data.table")
library("rlist")
library("yaml")

library("rpart")
library("parallel")

#paquetes necesarios para la Bayesian Optimization
library("DiceKriging")
library("mlrMBO")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,615863,32.9,1335898,71.4,1335898,71.4
Vcells,1155927,8.9,8388608,64.0,1801047,13.8


Loading required package: mlr

Loading required package: ParamHelpers

Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to the focus on 'mlr3' there might be
uncaught bugs meanwhile in {mlr} - please consider switching.

Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames




In [3]:
#para poder usarlo en la PC y en la nube
switch ( Sys.info()[['sysname']],
         Windows = { directory.root   <-  "M:\\" },   #Microsoft Windows
         Darwin  = { directory.root   <-  "~/dm/" },  #Apple MAC
#         Linux   = { directory.root   <-  "~/buckets/b1/" }  #Entorno Google Cloud
         Linux   = { directory.root   <-  "~/" }  #Hay un problema en el almacenamiento en cloud del BO por el momento se graba en el usuario
       )
#defino la carpeta donde trabajo

#Hay un error en el almacenamiento 

setwd( "/home/devsar/Documents/Austral/Laboratorio de Implementación" )

In [4]:
kexperimento  <- 301   #NA si se corre la primera vez, un valor concreto si es para continuar procesando

kscript           <- "04.02 - BO Rpart FE"
kBO_iter    <-  400   #cantidad de iteraciones de la Optimizacion Bayesiana

kcarpeta_datasets    <- "./datasets/"                          #VM o Ubuntu

#Archivo con datos etiquetados para entrenamiento
karchivo_entrada      <-  paste0(kcarpeta_datasets, "competencia1_2022_FE_4.csv")


#Ganancia por TP
kTPGain               <-  78000

#Pérdida por FP
kFPGain               <-  -2000


hs  <- makeParamSet(
          makeNumericParam("cp"       , lower= -1   , upper=    0.01),
          makeIntegerParam("minsplit" , lower=  300L  , upper= 2000L),  #la letra L al final significa ENTERO
          makeIntegerParam("minbucket", lower=  3L  , upper= 1000L),
          makeIntegerParam("maxdepth" , lower=  6L  , upper=   14L),
          forbidden = quote( minbucket > 0.5*minsplit ) )


ksemilla_azar  <- 700027

#------------------------------------------------------------------------------
#Funcion que lleva el registro de los experimentos

get_experimento  <- function()
{
  if( !file.exists( "./maestro.yaml" ) )  cat( file="./maestro.yaml", "experimento: 1000" )

  exp  <- read_yaml( "./maestro.yaml" )
  experimento_actual  <- exp$experimento

  exp$experimento  <- as.integer(exp$experimento + 1)
  Sys.chmod( "./maestro.yaml", mode = "0644", use_umask = TRUE)
  write_yaml( exp, "./maestro.yaml" )
  Sys.chmod( "./maestro.yaml", mode = "0444", use_umask = TRUE) #dejo el archivo readonly

  return( experimento_actual )
}

In [5]:
#------------------------------------------------------------------------------
#Funcion que lleva el registro de los experimentos

get_experimento  <- function()
{
  if( !file.exists( "./maestro.yaml" ) )  cat( file="./maestro.yaml", "experimento: 1000" )

  exp  <- read_yaml( "./maestro.yaml" )
  experimento_actual  <- exp$experimento

  exp$experimento  <- as.integer(exp$experimento + 1)
  Sys.chmod( "./maestro.yaml", mode = "0644", use_umask = TRUE)
  write_yaml( exp, "./maestro.yaml" )
  Sys.chmod( "./maestro.yaml", mode = "0444", use_umask = TRUE) #dejo el archivo readonly

  return( experimento_actual )
}

In [6]:
#------------------------------------------------------------------------------
#funcion para particionar, es la que Andres reemplaza con caret

particionar  <- function( data, division, agrupa="", campo="fold", start=1, seed=NA )
{
  if( !is.na( seed)  )   set.seed( seed )

  bloque  <- unlist( mapply(  function(x,y) { rep( y, x ) }, division, seq( from=start, length.out=length(division) )  ) )

  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))) )[1:.N],
           by= agrupa ]
}
#------------------------------------------------------------------------------

In [7]:
ArbolSimple  <- function( fold_test, data, param )
{
  variables = ". -Master_delinquency-Master_status-Master_mfinanciacion_limite-Master_Fvencimiento-Master_Finiciomora-Master_msaldototal-Master_msaldopesos-Master_msaldodolares-Master_mconsumospesos-Master_mconsumosdolares-Master_mlimitecompra-Master_madelantopesos-Master_madelantodolares-Master_fultimo_cierre-Master_mpagado-Master_mpagospesos-Master_mpagosdolares-Master_fechaalta-Master_mconsumototal-Master_cconsumos-Master_cadelantosefectivo-Master_mpagominimo-Visa_delinquency-Visa_status-Visa_mfinanciacion_limite-Visa_Fvencimiento-Visa_Finiciomora-Visa_msaldototal-Visa_msaldopesos-Visa_msaldodolares-Visa_mconsumospesos-Visa_mconsumosdolares-Visa_mlimitecompra-Visa_madelantopesos-Visa_madelantodolares-Visa_fultimo_cierre-Visa_mpagado-Visa_mpagospesos-Visa_mpagosdolares-Visa_fechaalta-Visa_mconsumototal-Visa_cconsumos-Visa_cadelantosefectivo-Visa_mpagominimo"
  my_formula = paste("clase_ternaria ~", variables)
  #genero el modelo
  modelo  <- rpart(my_formula, 
                   data= data[ fold != fold_test, ],
                   xval= 0,
                   control= param )

  #aplico el modelo a los datos de testing, fold==2
  prediccion  <- predict( modelo, data[ fold==fold_test, ], type = "prob")

  prob_baja2  <- prediccion[, "BAJA+2"]

  ganancia_testing  <- sum(  data[ fold==fold_test ][ prob_baja2 >0.025,  ifelse( clase_ternaria=="BAJA+2", kTPGain, kFPGain ) ] )

  return( ganancia_testing )
}

In [8]:
#------------------------------------------------------------------------------

ArbolesCrossValidation  <- function( data, param, qfolds, pagrupa, semilla )
{
  divi  <- rep( 1, qfolds )
  particionar( data, divi, seed=semilla, agrupa=pagrupa )

  ganancias  <- mcmapply( ArbolSimple, 
                          seq(qfolds), # 1 2 3 4 5  
                          MoreArgs= list( data, param), 
                          SIMPLIFY= FALSE,
                          mc.cores= 5 )   #se puede subir a 5 si posee Linux o Mac OS

  data[ , fold := NULL ]
  #devuelvo la primer ganancia y el promedio
  return( mean( unlist( ganancias )) *  qfolds )   #aqui normalizo la ganancia
}

In [9]:
#------------------------------------------------------------------------------
#esta funcion solo puede recibir los parametros que se estan optimizando
#el resto de los parametros se pasan como variables globales

EstimarGanancia  <- function( x )
{
   GLOBAL_iteracion  <<-  GLOBAL_iteracion + 1

   xval_folds  <- 5
   ganancia  <-  ArbolesCrossValidation( dtrain, param=x, qfolds= xval_folds, pagrupa="clase_ternaria", semilla=ksemilla_azar )

   #si tengo una ganancia superadora, genero el archivo para Kaggle
   if(  ganancia > GLOBAL_ganancia_max )
   {
     GLOBAL_ganancia_max <<-  ganancia  #asigno la nueva maxima ganancia
    
     variables = ". -Master_delinquency-Master_status-Master_mfinanciacion_limite-Master_Fvencimiento-Master_Finiciomora-Master_msaldototal-Master_msaldopesos-Master_msaldodolares-Master_mconsumospesos-Master_mconsumosdolares-Master_mlimitecompra-Master_madelantopesos-Master_madelantodolares-Master_fultimo_cierre-Master_mpagado-Master_mpagospesos-Master_mpagosdolares-Master_fechaalta-Master_mconsumototal-Master_cconsumos-Master_cadelantosefectivo-Master_mpagominimo-Visa_delinquency-Visa_status-Visa_mfinanciacion_limite-Visa_Fvencimiento-Visa_Finiciomora-Visa_msaldototal-Visa_msaldopesos-Visa_msaldodolares-Visa_mconsumospesos-Visa_mconsumosdolares-Visa_mlimitecompra-Visa_madelantopesos-Visa_madelantodolares-Visa_fultimo_cierre-Visa_mpagado-Visa_mpagospesos-Visa_mpagosdolares-Visa_fechaalta-Visa_mconsumototal-Visa_cconsumos-Visa_cadelantosefectivo-Visa_mpagominimo"
     my_formula = paste("clase_ternaria ~", variables)
     modelo  <- rpart(my_formula,
                      data= dtrain,
                      xval= 0,
                      control= x )

     #genero el vector con la prediccion, la probabilidad de ser positivo
     prediccion  <- predict( modelo, dapply)

     prob_baja2  <- prediccion[, "BAJA+2"]
     Predicted   <- ifelse( prob_baja2 > 0.025, 1, 0 )

     entrega  <-  as.data.table( list( "numero_de_cliente"=dapply$numero_de_cliente, "Predicted"=Predicted)  )

     #genero el archivo para Kaggle
     fwrite( entrega, 
             file= paste0(kkaggle, GLOBAL_iteracion, ".csv" ),
             sep=  "," )
   }

   return( ganancia )
}
#------------------------------------------------------------------------------

In [10]:
#Aqui empieza el programa

if( is.na(kexperimento ) )   kexperimento <- get_experimento()  #creo el experimento

In [11]:
#en estos archivos quedan los resultados
kbayesiana  <- paste0("./E",  kexperimento, "_rpart.RDATA" )
kkaggle     <- paste0("./E",kexperimento, "_rpart_kaggle_" )

In [15]:
GLOBAL_ganancia_max  <-  -Inf
GLOBAL_iteracion  <- 19

In [12]:
#cargo los datos
dataset <- fread(karchivo_entrada)

#Para hacer pruebas rapidas puedo reducir el dataset a una fraccion
#subsample <- sample(1:nrow(dataset), .1 * nrow(dataset))
#dataset <- dataset[subsample,]

dtrain  <- dataset[ foto_mes==202101 ]  #defino donde voy a entrenar

dapply  <- dataset[ foto_mes==202103 ]  #defino donde voy a aplicar el modelo

In [13]:
#Aqui comienza la configuracion de la Bayesian Optimization

configureMlr( show.learner.output = FALSE)

funcion_optimizar  <- EstimarGanancia

#configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
#por favor, no desesperarse por lo complejo
obj.fun  <- makeSingleObjectiveFunction(
              fn=       funcion_optimizar,
              minimize= FALSE,   #estoy Maximizando la ganancia
              noisy=    TRUE,
              par.set=  hs,
              has.simple.signature = FALSE
             )

ctrl  <- makeMBOControl( save.on.disk.at.time= 60,  save.file.path= kbayesiana)
ctrl  <- setMBOControlTermination(ctrl, iters= kBO_iter )
ctrl  <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

surr.km  <-  makeLearner("regr.km", predict.type= "se", covtype= "matern3_2", control= list(trace= TRUE))

In [16]:
#inicio la optimizacion bayesiana
if(!file.exists(kbayesiana)) {
  run  <- mbo(obj.fun, learner = surr.km, control = ctrl)
} else  run  <- mboContinue( kbayesiana )   #retomo en caso que ya exista

[mbo] 19: cp=-0.012; minsplit=1092; minbucket=25; maxdepth=6 : y = 2.06e+07 : 39.5 secs : infill_ei

Saved the current state after iteration 20 in the file ./E301_rpart.RDATA.

[mbo] 20: cp=-0.925; minsplit=1083; minbucket=273; maxdepth=6 : y = 2.01e+07 : 21.1 secs : infill_ei

[mbo] 21: cp=-0.993; minsplit=1101; minbucket=352; maxdepth=8 : y = 1.97e+07 : 34.3 secs : infill_ei

[mbo] 22: cp=-0.983; minsplit=1476; minbucket=29; maxdepth=14 : y = 1.85e+07 : 50.7 secs : infill_ei

Saved the current state after iteration 23 in the file ./E301_rpart.RDATA.

[mbo] 23: cp=-0.118; minsplit=1454; minbucket=172; maxdepth=7 : y = 2.01e+07 : 28.9 secs : infill_ei

[mbo] 24: cp=-0.32; minsplit=1090; minbucket=545; maxdepth=14 : y = 1.91e+07 : 48.5 secs : infill_ei

Saved the current state after iteration 25 in the file ./E301_rpart.RDATA.

[mbo] 25: cp=-0.949; minsplit=1522; minbucket=319; maxdepth=6 : y = 1.97e+07 : 26.4 secs : infill_ei

[mbo] 26: cp=-0.33; minsplit=1250; minbucket=374; maxdepth=

In [ ]:
quit( save="no" )